# Исследование надежности заемщиков


## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

По результатам исследования необходимо ответить на следующие вопросы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

## Импорт библиотек, открытие файла, первичный обзор данных

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer

In [2]:
# Открытие файла с данными
try:
    data = pd.read_csv('data.csv')
except:
    data = pd.read_csv('/datasets/data.csv')

In [3]:
# Получение общей информации о таблице и данных в ней
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Из полученной информации видно, что общий размер таблицы 21525 х 12. Названия столбцов корректны. Тип данных в столбцах days_employed и total_income можно сменить на int. Первый может быть только целочисленный, а во втором избыточная точность не нужна. Остальные данные выглядят корректными и похожи на описание от Заказчика

Описание данных от Заказчика
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Предобработка данных

### Определение явных пропусков в таблице

In [5]:
# вывод кол-ва явных пропусков во всей таблице
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [6]:
# выведем первые и последние 5 строк с пропусками
display(data[data.days_employed.isna() | data.total_income.isna()])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


По данной выборке нельзя однозначно сказать о причине пропусков. Возможен и технический и человеческий фактор. Возможно, что данные поля были необязательны для заполнения.

In [7]:
# заменяем отрицательные значения положительными
# пишем функцию для замены
def neg_poz(item):
    if item < 0:
        item *= -1
        return item
    else:
        return item
# заменяем данные
data.days_employed = data.days_employed.apply(neg_poz)
# проверяем
data[data.days_employed < 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [8]:
# заменяем пропуски в стаже на медианные значения
days_employed_median = data.days_employed.median()
data.days_employed = data.days_employed.fillna(days_employed_median)
# проверяем
data.days_employed.isna().sum()

0

In [9]:
# проверяем тип данных в столбце days_employed
type(data.days_employed[0])

numpy.float64

In [10]:
# заменяем тип данных с float на int
data.days_employed = data.days_employed.astype('int')
# проверяем
type(data.days_employed[0])

numpy.int64

### Определение дубликатов в остальных столбцах

#### Столбец children

In [11]:
# информация о значениях в столбце children
display(data.children.value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Странные значения -1 и 20. -1 возможно техническое значение пропуска. Необходимо оценить возможность соединить значения с 0. А 20 слишком странное значение потому что нет значений в диапозоне от 6 до 19. Возможно это человеческий фактор. Возможно это значения либо 0 либо 2. Необходимо проанализировать похожи ли средние значения для слияния

In [12]:
# выводим размер таблицы
print(data.shape)
# процентное соотношение кол-ва строк -1 и 20 к общему числу
print('Процент значений -1 к общему кол-ву значений в таблице:', round(47 / data.shape[0] * 100, 2))
print('Процент значений 20 к общему кол-ву значений в таблице:', round(76 / data.shape[0] * 100, 2))

(21525, 12)
Процент значений -1 к общему кол-ву значений в таблице: 0.22
Процент значений 20 к общему кол-ву значений в таблице: 0.35


Так как оба значения составляют меньше процента от общего кол-ва значений в таблице, то позволим себе их удалить из выборки

In [13]:
data = data[(data.children != -1) & (data.children != 20)]
# Проверяем
display(data.children.value_counts())

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

#### Столбец dob_years

In [14]:
display(data.dob_years.value_counts())

35    614
40    603
41    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
44    543
29    543
30    536
48    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
57    457
53    457
51    446
59    441
55    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    263
23    252
65    194
22    183
66    183
67    167
21    110
0     100
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Все данные в рамках нормы. значения возраста 0 - техническое значение для пропуска. Так как в рамках исследования возраст нам не важен, то можно оставить все как есть

#### Столбецы education и education_id

In [15]:
display(data.education.value_counts())

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [16]:
display(data.education_id.value_counts())

1    15136
0     5237
2      741
3      282
4        6
Name: education_id, dtype: int64

Задачи:
- удалить дубликаты в education
- понять соответствие между education и education_id

In [17]:
# определяем к какому id относится какое образование
display(data[data.education_id == 0].sample(5))
display(data[data.education_id == 1].sample(5))
display(data[data.education_id == 2].sample(5))
display(data[data.education_id == 3].sample(5))
display(data[data.education_id == 4].sample(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21230,0,3249,65,высшее,0,женат / замужем,0,M,компаньон,0,310497.399045,ремонт жилью
8919,0,1983,30,высшее,0,гражданский брак,1,F,сотрудник,0,510808.469533,на проведение свадьбы
20832,1,4627,31,высшее,0,гражданский брак,1,F,госслужащий,0,271685.087653,на проведение свадьбы
19907,4,3452,29,высшее,0,женат / замужем,0,F,сотрудник,0,241089.448393,операции с коммерческой недвижимостью
6784,0,1033,29,высшее,0,Не женат / не замужем,4,M,сотрудник,1,167869.467203,получение образования


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15206,0,6777,48,среднее,1,гражданский брак,1,F,сотрудник,0,115185.442754,на проведение свадьбы
8852,0,368789,60,среднее,1,женат / замужем,0,F,пенсионер,0,83104.088771,профильное образование
3116,0,2574,52,среднее,1,женат / замужем,0,M,сотрудник,0,150626.097588,операции с коммерческой недвижимостью
15905,2,2194,31,среднее,1,женат / замужем,0,M,компаньон,0,NaN,свой автомобиль
13655,0,12235,57,среднее,1,женат / замужем,0,F,компаньон,0,145630.879196,покупка недвижимости


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2711,1,4342,30,неоконченное высшее,2,гражданский брак,1,F,сотрудник,0,160686.862925,операции с недвижимостью
18331,0,2194,39,неоконченное высшее,2,женат / замужем,0,M,сотрудник,0,NaN,строительство собственной недвижимости
4365,0,310,23,неоконченное высшее,2,гражданский брак,1,F,сотрудник,0,107431.442793,автомобили
17029,1,273,38,неоконченное высшее,2,женат / замужем,0,M,сотрудник,0,243450.676212,покупка недвижимости
3681,0,1209,31,неоконченное высшее,2,Не женат / не замужем,4,F,сотрудник,0,390062.861144,покупка жилой недвижимости


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6772,0,3878,54,начальное,3,в разводе,3,F,сотрудник,0,71563.196686,автомобиль
1038,0,4586,44,начальное,3,гражданский брак,1,F,сотрудник,0,89987.147789,сыграть свадьбу
20870,1,2240,26,начальное,3,женат / замужем,0,M,сотрудник,0,175780.973093,покупка жилья для сдачи
21170,2,181,38,начальное,3,женат / замужем,0,M,сотрудник,0,175649.780503,покупка коммерческой недвижимости
17696,1,2194,45,начальное,3,женат / замужем,0,F,пенсионер,0,NaN,операции с недвижимостью


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12786,0,376276,62,ученая степень,4,женат / замужем,0,F,пенсионер,0,255425.196556,покупка жилой недвижимости
2963,0,337584,69,Ученая степень,4,женат / замужем,0,M,пенсионер,0,98752.495442,покупка жилой недвижимости
6551,0,5352,58,ученая степень,4,женат / замужем,0,M,сотрудник,0,268411.214536,заняться высшим образованием
21519,1,2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
12021,3,5968,36,ученая степень,4,женат / замужем,0,F,госслужащий,0,111392.231107,покупка жилья


0 - высшее, 1 - среднее, 2 - неоконченное высшее, 3 - начальное, 4 - ученая степень. Так как образование в рамках исследования нам не важно, то можно соединить ученую степень с высшим образованием. Получить степень можно только получения высшего кол-во степеней всего 6, высших 5260. Также внести необходимые изменения в education_id

In [18]:
# приводим все значения education_id к нижнему регистру
data.education = data.education.str.lower()
# проверяем
display(data.education.value_counts())

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

In [19]:
# включаем значения "ученая степень" в "высшее" и убираем лишний id из education_id
data.loc[data.education == 'ученая степень', 'education'] = 'высшее'
data.loc[data.education_id == 4, 'education_id'] = 0
# проверяем
display(data.education.value_counts())
display(data.education_id.value_counts())

среднее                15136
высшее                  5243
неоконченное высшее      741
начальное                282
Name: education, dtype: int64

1    15136
0     5243
2      741
3      282
Name: education_id, dtype: int64

#### Столбцы family_status и family_status_id

In [20]:
display(data.family_status.value_counts())
display(data.family_status_id.value_counts())

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

0    12302
1     4160
4     2799
3     1189
2      952
Name: family_status_id, dtype: int64

In [21]:
# определяем к какому id относится какой семейный статус
display(data[data.family_status_id == 0].sample(5))
display(data[data.family_status_id == 1].sample(5))
display(data[data.family_status_id == 2].sample(5))
display(data[data.family_status_id == 3].sample(5))
display(data[data.family_status_id == 4].sample(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
19060,1,2477,40,высшее,0,женат / замужем,0,F,компаньон,1,165159.222869,операции с недвижимостью
17153,2,3607,33,среднее,1,женат / замужем,0,F,сотрудник,0,196904.724368,приобретение автомобиля
6507,1,2008,46,среднее,1,женат / замужем,0,M,сотрудник,0,116751.943953,покупка жилья для сдачи
7420,0,2194,58,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,сделка с подержанным автомобилем
13524,0,341,48,среднее,1,женат / замужем,0,M,сотрудник,0,190917.856018,ремонт жилью


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
13654,2,2194,40,высшее,0,гражданский брак,1,F,компаньон,0,NaN,на покупку своего автомобиля
20465,0,241,26,высшее,0,гражданский брак,1,F,компаньон,0,144171.244792,на проведение свадьбы
3856,0,1584,42,среднее,1,гражданский брак,1,F,сотрудник,0,107632.581916,на проведение свадьбы
6582,0,3349,56,среднее,1,гражданский брак,1,M,сотрудник,0,167186.661839,сыграть свадьбу
20903,0,3222,28,высшее,0,гражданский брак,1,F,госслужащий,0,174622.608484,на проведение свадьбы


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6558,0,1871,57,высшее,0,вдовец / вдова,2,F,госслужащий,0,184722.741054,строительство недвижимости
15015,0,359520,51,высшее,0,вдовец / вдова,2,F,пенсионер,0,77369.057129,покупка жилья
6142,0,395141,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,187942.844583,на покупку автомобиля
14455,0,371325,55,среднее,1,вдовец / вдова,2,F,пенсионер,0,84559.529899,операции с недвижимостью
6487,0,10691,62,среднее,1,вдовец / вдова,2,F,сотрудник,0,90894.688311,покупка жилой недвижимости


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1848,1,531,38,высшее,0,в разводе,3,F,сотрудник,0,121849.505298,заняться высшим образованием
19982,0,165,56,высшее,0,в разводе,3,F,сотрудник,0,124544.845827,жилье
21035,0,874,51,среднее,1,в разводе,3,M,сотрудник,0,79799.148961,ремонт жилью
12724,3,2291,34,среднее,1,в разводе,3,F,сотрудник,0,124239.651802,покупка жилья
5072,0,2194,43,высшее,0,в разводе,3,M,компаньон,0,NaN,автомобили


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
11206,1,13065,57,среднее,1,Не женат / не замужем,4,F,сотрудник,0,295357.813136,недвижимость
12995,0,401,27,высшее,0,Не женат / не замужем,4,M,компаньон,0,111913.334479,недвижимость
7312,0,600,31,среднее,1,Не женат / не замужем,4,M,сотрудник,0,158712.685873,операции с жильем
8056,0,355,62,неоконченное высшее,2,Не женат / не замужем,4,F,компаньон,0,253630.273986,жилье
8388,0,210,22,неоконченное высшее,2,Не женат / не замужем,4,F,госслужащий,1,50962.678837,покупка жилья


0 - женат / замужем, 1 - гражданский брак, 2 - вдовец / вдова, 3 - в разводе, 4 - Не женат / не замужем. В целом разделение и данные устраивают. Оставляем все как есть

#### Столбец gender

In [22]:
display(data.gender.value_counts())

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

Одно странное значение XNA. Удаляем строку

In [23]:
data = data[data.gender != 'XNA']
# проверяем
display(data.gender.value_counts())

F    14154
M     7247
Name: gender, dtype: int64

#### Столбец debt

In [24]:
display(data.debt.value_counts())

0    19669
1     1732
Name: debt, dtype: int64

Нормальные значения. Оставляем как есть

#### Столбец income_type

In [25]:
display(data.income_type.value_counts())

сотрудник          11050
компаньон           5053
пенсионер           3839
госслужащий         1453
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Безработных, предпринимателей, в декрете и студентов слишком малое кол-во. Удаляем данные строки

In [26]:
data = data[(data.income_type != 'безработный') & (data.income_type != 'предприниматель') & (data.income_type != 'в декрете') & 
           (data.income_type != 'студент')]
# проверяем
display(data.income_type.value_counts())

сотрудник      11050
компаньон       5053
пенсионер       3839
госслужащий     1453
Name: income_type, dtype: int64

In [27]:
# вычисляем медиану в столбце total_income
total_income_median = data.total_income.median()
# заполняем пропуски медианными значениями
data.total_income = data.total_income.fillna(total_income_median)
# проверям
data.total_income.isna().sum()

0

In [28]:
# проверяем тип данных в столбце total_income
type(data.total_income[0])

numpy.float64

In [29]:
# изменяем тип данных на int
data.total_income = data.total_income.astype('int')
# проверяем
type(data.total_income[0])

numpy.int64

In [30]:
# duplicated для всей таблицы
data[data.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194,41,среднее,1,женат / замужем,0,F,сотрудник,0,145020,покупка жилья для семьи
3290,0,2194,58,среднее,1,гражданский брак,1,F,пенсионер,0,145020,сыграть свадьбу
4182,1,2194,34,высшее,0,гражданский брак,1,F,сотрудник,0,145020,свадьба
4851,0,2194,60,среднее,1,гражданский брак,1,F,пенсионер,0,145020,свадьба
5557,0,2194,58,среднее,1,гражданский брак,1,F,пенсионер,0,145020,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,2194,64,среднее,1,женат / замужем,0,F,пенсионер,0,145020,дополнительное образование
21032,0,2194,60,среднее,1,женат / замужем,0,F,пенсионер,0,145020,заняться образованием
21132,0,2194,47,среднее,1,женат / замужем,0,F,сотрудник,0,145020,ремонт жилью
21281,1,2194,30,высшее,0,женат / замужем,0,F,сотрудник,0,145020,покупка коммерческой недвижимости


#### Леммантизация столбца purpose

In [31]:
# создадим пустой список для уникальных значений из purpose
data_purpose = []
# заполним список
for item in data.purpose:
    if item not in data_purpose:
        data_purpose.append(item)
# выведем список уникальных целей для кредита
print(data_purpose)

['покупка жилья', 'приобретение автомобиля', 'дополнительное образование', 'сыграть свадьбу', 'операции с жильем', 'образование', 'на проведение свадьбы', 'покупка жилья для семьи', 'покупка недвижимости', 'покупка коммерческой недвижимости', 'покупка жилой недвижимости', 'строительство собственной недвижимости', 'недвижимость', 'строительство недвижимости', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'операции с коммерческой недвижимостью', 'строительство жилой недвижимости', 'жилье', 'операции со своей недвижимостью', 'автомобили', 'заняться образованием', 'сделка с подержанным автомобилем', 'получение образования', 'автомобиль', 'свадьба', 'получение дополнительного образования', 'покупка своего жилья', 'операции с недвижимостью', 'получение высшего образования', 'свой автомобиль', 'сделка с автомобилем', 'профильное образование', 'высшее образование', 'покупка жилья для сдачи', 'на покупку автомобиля', 'ремонт жилью', 'заняться высшим образованием']


In [32]:
# создаем экземпляр класса
m = Mystem()
# создаем строку на основе списка уникальных значений
data_purpose_str = ' '.join(data_purpose)
# леммантизируем
lemmas = m.lemmatize(data_purpose_str)
# выводим результат
print(lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

Не сложно заметить следующие группы слов

In [33]:
lemmas_house = ['жилье', 'недвижимость', 'строительство', 'ремонт']
lemmas_edu = ['образование']
lemmas_wed = ['свадьба']
lemmas_auto = ['автомобиль']

С помощью стемминга найдем основы слов для дальнейшей категоризации purpose

In [34]:
data_purpose = []
for item in data.purpose:
    temps = item.split()
    for temp in temps:
        if temp not in data_purpose:
            data_purpose.append(temp)

data_purpose.sort()
print(data_purpose)

['автомобилем', 'автомобили', 'автомобиль', 'автомобиля', 'высшего', 'высшее', 'высшим', 'для', 'дополнительного', 'дополнительное', 'жилой', 'жилье', 'жильем', 'жилью', 'жилья', 'заняться', 'коммерческой', 'на', 'недвижимости', 'недвижимость', 'недвижимостью', 'образование', 'образованием', 'образования', 'операции', 'подержанного', 'подержанным', 'покупка', 'покупку', 'получение', 'приобретение', 'проведение', 'профильное', 'ремонт', 'с', 'свадьба', 'свадьбу', 'свадьбы', 'своего', 'своей', 'свой', 'сдачи', 'сделка', 'семьи', 'со', 'собственной', 'строительство', 'сыграть']


In [35]:
# создаем экземпляр класса с русским языком
russian_stemmer = SnowballStemmer('russian')

In [36]:
data_stem = []
for word in data_purpose:
    temp = russian_stemmer.stem(word)
    if temp not in data_stem:
        data_stem.append(temp)
        
print(data_stem)

['автомобил', 'автомоб', 'высш', 'для', 'дополнительн', 'жил', 'заня', 'коммерческ', 'на', 'недвижим', 'образован', 'операц', 'подержа', 'покупк', 'получен', 'приобретен', 'проведен', 'профильн', 'ремонт', 'с', 'свадьб', 'сво', 'сдач', 'сделк', 'сем', 'со', 'собствен', 'строительств', 'сыгра']


In [37]:
stem_house = ['жил', 'недвижим']
stem_edu = ['высш', 'образован']
stem_wed = ['свадьб']
stem_auto = ['автомоб', 'автомобил']

In [38]:
# пишем функцию для категоризации стоблца purpose
def purpose_cat_range(items):
    for item in items.split():
        stemmed_item = russian_stemmer.stem(item)
        if stemmed_item in stem_house:
            return 0
        if stemmed_item in stem_edu:
            return 1
        if stemmed_item in stem_wed:
            return 2
        if stemmed_item in stem_auto:
            return 3

In [39]:
# 0 - недвижимость, 1 - образование, 2 - свадьба, 3 - авто
data['purpose_id'] = data['purpose'].apply(purpose_cat_range)

In [40]:
# проверка
display(data[data.purpose_id == 0].sample(10))
display(data[data.purpose_id == 1].sample(10))
display(data[data.purpose_id == 2].sample(10))
display(data[data.purpose_id == 3].sample(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id
8119,0,362036,59,высшее,0,женат / замужем,0,F,пенсионер,0,183560,покупка жилья,0
8450,2,973,45,среднее,1,гражданский брак,1,M,сотрудник,0,157646,жилье,0
19177,2,1803,36,среднее,1,женат / замужем,0,F,сотрудник,0,163292,строительство собственной недвижимости,0
2466,0,1160,62,высшее,0,вдовец / вдова,2,F,компаньон,0,321164,покупка жилья,0
21273,0,196,44,высшее,0,женат / замужем,0,M,компаньон,0,372416,ремонт жилью,0
6547,1,1253,50,среднее,1,вдовец / вдова,2,F,сотрудник,0,251569,покупка своего жилья,0
16153,1,2194,21,среднее,1,женат / замужем,0,F,госслужащий,0,145020,операции с коммерческой недвижимостью,0
13898,0,340038,63,среднее,1,женат / замужем,0,F,пенсионер,0,60633,покупка своего жилья,0
2853,0,4989,64,среднее,1,женат / замужем,0,M,компаньон,0,247669,покупка жилья,0
6649,0,345409,72,среднее,1,вдовец / вдова,2,F,пенсионер,0,115678,недвижимость,0


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id
5450,1,2504,51,среднее,1,женат / замужем,0,F,сотрудник,0,67626,получение дополнительного образования,1
11795,0,384734,58,среднее,1,женат / замужем,0,F,пенсионер,0,161698,образование,1
19693,0,331,60,среднее,1,женат / замужем,0,F,сотрудник,0,110500,дополнительное образование,1
21431,0,1656,27,высшее,0,женат / замужем,0,F,сотрудник,0,187354,получение дополнительного образования,1
18039,0,448,50,среднее,1,в разводе,3,F,компаньон,0,129289,получение высшего образования,1
5325,0,363791,66,среднее,1,женат / замужем,0,F,пенсионер,0,161365,получение дополнительного образования,1
39,0,650,31,высшее,0,гражданский брак,1,F,компаньон,0,754240,заняться образованием,1
20984,0,2194,67,высшее,0,женат / замужем,0,F,пенсионер,0,145020,заняться высшим образованием,1
16253,0,329460,53,среднее,1,гражданский брак,1,F,пенсионер,0,94973,заняться высшим образованием,1
11656,0,643,24,среднее,1,Не женат / не замужем,4,M,сотрудник,0,166159,заняться образованием,1


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id
514,2,2369,45,неоконченное высшее,2,гражданский брак,1,F,сотрудник,0,120133,на проведение свадьбы,2
6786,0,4236,44,среднее,1,гражданский брак,1,F,сотрудник,0,118284,сыграть свадьбу,2
1934,0,5044,55,среднее,1,гражданский брак,1,F,сотрудник,0,100997,сыграть свадьбу,2
1113,2,446,31,среднее,1,гражданский брак,1,M,сотрудник,0,405790,свадьба,2
11884,0,425,37,среднее,1,гражданский брак,1,F,сотрудник,0,174501,сыграть свадьбу,2
14401,0,1265,26,высшее,0,гражданский брак,1,F,госслужащий,0,330682,свадьба,2
16033,0,2194,53,высшее,0,гражданский брак,1,F,пенсионер,0,145020,на проведение свадьбы,2
14900,0,11274,59,среднее,1,гражданский брак,1,F,сотрудник,0,95256,сыграть свадьбу,2
14586,0,4391,42,высшее,0,гражданский брак,1,F,сотрудник,0,166471,сыграть свадьбу,2
11197,1,1239,23,среднее,1,гражданский брак,1,F,сотрудник,0,46195,свадьба,2


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id
12626,0,1722,31,среднее,1,женат / замужем,0,F,сотрудник,0,136960,сделка с автомобилем,3
4955,1,1269,30,среднее,1,женат / замужем,0,F,сотрудник,0,120202,свой автомобиль,3
20063,0,347919,63,среднее,1,женат / замужем,0,F,пенсионер,0,98530,свой автомобиль,3
10426,0,176,34,среднее,1,гражданский брак,1,F,сотрудник,1,35249,на покупку подержанного автомобиля,3
18486,2,2480,45,высшее,0,женат / замужем,0,M,компаньон,0,112007,на покупку своего автомобиля,3
14046,1,1827,46,среднее,1,женат / замужем,0,M,компаньон,0,130966,свой автомобиль,3
3718,0,492,32,неоконченное высшее,2,гражданский брак,1,F,компаньон,0,129250,сделка с автомобилем,3
922,0,346393,59,среднее,1,женат / замужем,0,F,пенсионер,0,53476,на покупку своего автомобиля,3
2622,0,549,25,среднее,1,Не женат / не замужем,4,F,сотрудник,0,159805,на покупку подержанного автомобиля,3
13259,0,187,29,среднее,1,Не женат / не замужем,4,M,сотрудник,0,165712,приобретение автомобиля,3


#### Категоризация

Столбцами для словарей можно выделить:
- education и education_id
- family_status и family_status_id
- income_type и income_type_id
- purpose и purpose_id

In [41]:
# edu_dict
edu_dict = data[['education_id', 'education']]
# удаляем дубликаты
edu_dict.education_id = edu_dict.education_id.drop_duplicates()
edu_dict = edu_dict.dropna().sort_values('education_id').reset_index(drop=True)
print(edu_dict)

   education_id            education
0           0.0               высшее
1           1.0              среднее
2           2.0  неоконченное высшее
3           3.0            начальное


/tmp/ipykernel_21366/510186334.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edu_dict.education_id = edu_dict.education_id.drop_duplicates()


In [42]:
edu_dict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   education_id  4 non-null      float64
 1   education     4 non-null      object 
dtypes: float64(1), object(1)
memory usage: 192.0+ bytes


In [43]:
edu_dict['education_id'] = edu_dict['education_id'].astype('int')

In [44]:
# fam_stat_dict
fam_stat_dict = data[['family_status_id', 'family_status']]
# удаляем дубликаты
fam_stat_dict['family_status_id'] = fam_stat_dict['family_status_id'].drop_duplicates()
fam_stat_dict = fam_stat_dict.dropna().sort_values('family_status_id').reset_index(drop=True)
fam_stat_dict['family_status_id'] = fam_stat_dict['family_status_id'].astype('int')

/tmp/ipykernel_21366/619619956.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fam_stat_dict['family_status_id'] = fam_stat_dict['family_status_id'].drop_duplicates()


In [45]:
# пишем функцию для категоризации столбца income_type
def income_type_range(income_type):
    if income_type == 'госслужащий':
        return 0
    if income_type == 'компаньон':
        return 1
    if income_type == 'пенсионер':
        return 2
    if income_type == 'сотрудник':
        return 3

In [46]:
# заполняем новый столбец
data['income_type_id'] = data['income_type'].apply(income_type_range)
# проверяем
display(data.sample(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id,income_type_id
1959,0,334048,47,среднее,1,женат / замужем,0,M,пенсионер,0,101677,приобретение автомобиля,3,2
2782,0,348232,50,среднее,1,женат / замужем,0,F,пенсионер,0,70345,строительство жилой недвижимости,0,2
10921,2,2248,37,высшее,0,женат / замужем,0,F,сотрудник,0,227189,сделка с автомобилем,3,3
7016,1,1207,47,среднее,1,Не женат / не замужем,4,F,сотрудник,0,115438,автомобили,3,3
19183,0,356542,58,среднее,1,вдовец / вдова,2,F,пенсионер,0,203425,строительство собственной недвижимости,0,2
1630,2,2194,42,среднее,1,женат / замужем,0,F,сотрудник,0,145020,покупка жилой недвижимости,0,3
13063,0,1357,28,высшее,0,женат / замужем,0,M,компаньон,0,102705,операции с коммерческой недвижимостью,0,1
17682,0,653,31,среднее,1,Не женат / не замужем,4,M,сотрудник,0,233363,операции с жильем,0,3
1498,1,3437,35,среднее,1,женат / замужем,0,F,компаньон,0,165145,заняться высшим образованием,1,1
2866,1,2521,30,среднее,1,гражданский брак,1,F,госслужащий,1,276740,свой автомобиль,3,0


In [47]:
# purpose_dict
purpose_data = [
    [0, 'жилье'],
    [1, 'образование'],
    [2, 'свадьба'],
    [3, 'автомобиль']
]
purpose_dict = pd.DataFrame(data=purpose_data, columns=['purpose_id', 'purpose'])
display(purpose_dict)

,purpose_id,purpose
0,0,жилье
1,1,образование
2,2,свадьба
3,3,автомобиль


In [48]:
# пишем функцию для категоризации столбца children
def children_range(children):
    if children == 0:
        return 0
    if 0 < children < 3:
        return 1
    if children >= 3:
        return 2

In [49]:
# добавляем в список столбцов новый
data['children_id'] = data.children.apply(children_range)
data.children_id = data.children_id.astype('int')
# children_dict
children_data = [
    [0, 'нет детей'],
    [1, '1 или 2 ребенка'],
    [2, 'многодетные']
]
children_dict = pd.DataFrame(data=children_data, columns=['children_id', 'children_count'])

In [50]:
# Необходима категоризация по доходу
# Посмотрим на данные столбца total_income
display(data['total_income'].describe())

count    2.139500e+04
mean     1.651776e+05
std      9.797384e+04
min      2.066700e+04
25%      1.077220e+05
50%      1.450200e+05
75%      1.955460e+05
max      2.265604e+06
Name: total_income, dtype: float64

Данные можно разбить по 4 большим группам
- до 107_000
- от 107_000 до 142_000
- от 142_000 до 195_000
- от 195_000

In [51]:
# напишем функцию для категоризации
def total_income_type_range(total_income):
    if total_income < 107_000:
        return 0
    if 107_000 <= total_income < 142_000:
        return 1
    if 142_000 <= total_income < 195_000:
        return 2
    if total_income >= 195_000:
        return 3

In [52]:
# заполняем новый столбец
data['total_income_type_id'] = data['total_income'].apply(total_income_type_range)
# total_income_dict
total_income_data = [
    [0, 'низкие'],
    [1, 'ниже среднего'],
    [2, 'выше среднего'],
    [3, 'высокие']
]
total_income_dict = pd.DataFrame(data=total_income_data, columns=['total_income_type_id', 'total_income_type'])

In [53]:
# Окончательный вид главной таблицы
data = data[['children_id', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type_id', 'debt', 'total_income_type_id', 'purpose_id']]
display(data)

,children_id,dob_years,education_id,family_status_id,gender,income_type_id,debt,total_income_type_id,purpose_id
0,1,42,0,0,F,3,0,3,0
1,1,36,1,0,F,3,0,1,3
2,0,33,1,0,M,3,0,2,0
3,2,32,1,0,M,3,0,3,1
4,0,53,1,1,F,2,0,2,2
...,...,...,...,...,...,...,...,...,...
21520,1,43,1,1,F,1,0,3,0
21521,0,67,1,0,F,2,0,2,3
21522,1,38,1,1,M,3,1,0,0
21523,2,38,1,0,M,3,1,3,3


 ## Ответы на поставленные Заказчиком вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

### 1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для расчета зависимости нам понадобится следующее:
- уникальное кол-во детей
- общее кол-во заемщиков по каждому уникальному кол-ву детей
- отношение кол-ва возвратов и невозвратов к общему кол-ву взятых кредитов

In [54]:
# добавляем к таблице словарь
data_child = data.merge(children_dict, how='left')
# создаем сводную таблицу
data_child_debt = data_child.pivot_table(values='debt', index='children_count')
# округляем значения debt и вычисляем %
data_child_debt.debt = data_child_debt.debt.round(3) * 100
# сортируем
data_child_debt = data_child_debt.sort_values(by='debt', ascending=True)
# переименновываем столбец
data_child_debt = data_child_debt.rename(columns={'debt': '% debt of total'})
data_child_debt

,% debt of total
children_count,
нет детей,7.5
многодетные,8.2
1 или 2 ребенка,9.3


Заемщики без детей чаще возвращают заемные средства. С ростом кол-ва детей процент невозврата увеличивается.

### 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [55]:
# добавляем к таблице словарь
data_fam = data.merge(fam_stat_dict, how='left')
# создаем сводную таблицу
data_fam_debt = data_fam.pivot_table(values='debt', index='family_status')
# округляем значения debt и вычисляем %
data_fam_debt.debt = data_fam_debt.debt.round(3) * 100
# сортируем
data_fam_debt = data_fam_debt.sort_values(by='debt', ascending=True)
# переименновываем столбец
data_fam_debt = data_fam_debt.rename(columns={'debt': '% debt of total'})
data_fam_debt

,% debt of total
family_status,
вдовец / вдова,6.6
в разводе,7.1
женат / замужем,7.5
гражданский брак,9.3
Не женат / не замужем,9.8


Самый высокий процент возврата в вдовцов/вдов. Незначительно меньше процент у женатых/замужем и в разводе. Процент невозврата высокий у категорий заемщиков - в гражданском браке и не замужем/ не женатых

### 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [56]:
# добавляем к таблице словарь
data_inc = data.merge(total_income_dict, how='left')
# создаем сводную таблицу
data_inc_debt = data_inc.pivot_table(values='debt', index='total_income_type')
# округляем значения debt и вычисляем %
data_inc_debt.debt = data_inc_debt.debt.round(3) * 100
# сортируем
data_inc_debt = data_inc_debt.sort_values(by='debt', ascending=True)
# переименновываем столбец
data_inc_debt = data_inc_debt.rename(columns={'debt': '% debt of total'})
data_inc_debt

,% debt of total
total_income_type,
высокие,7.2
низкие,8.0
выше среднего,8.6
ниже среднего,8.6


От уровня дохода процент возвратов слабо зависит. Процент возврата несколько выше у заемщиков с высокими доходами

### 4. Как разные цели кредита влияют на его возврат в срок?

In [57]:
# добавляем к таблице словарь
data_pur = data.merge(purpose_dict, how='left')
# создаем сводную таблицу
data_pur_debt = data_pur.pivot_table(values='debt', index='purpose')
# округляем значения debt и вычисляем %
data_pur_debt.debt = data_pur_debt.debt.round(3) * 100
# сортируем
data_pur_debt = data_pur_debt.sort_values(by='debt', ascending=True)
# переименновываем столбец
data_pur_debt = data_pur_debt.rename(columns={'debt': '% debt of total'})
data_pur_debt

,% debt of total
purpose,
жилье,7.2
свадьба,7.8
образование,9.2
автомобиль,9.3


Кредиты чаще возвращают, взяв на покупку жилья и на проведение свадеб. И чаще не возвращают взяв кредит на автомобиль и образование

## Выводы

Общий вывод:
Однозначная зависимость на возврат кредита имеется в целях, на которые берут. Чаще возвращают кредиты на жилье и свадьбы и наоборот чаще не возвращают на автомобили и недвижимость.

Также однозначная зависимость существует в семейном положении. Чаще возвращают вдовы и вдовцы, а также заемщики в разводе.
Незначительно отличаются резальтаты у замужних и женатых. Чаще не возвращают заемщики находящиеся в гражданском браке и неженатые/незамужние

Доходы заемщиков почти не влияют на процент невозвратов. Хотя чуть меньше возвращают заемщики со средним достатком.

С увеличением кол-ва детей растет процент не возврата заемных средств.